<a href="https://colab.research.google.com/github/Sreekar-Kandhadai/pyspark-interview-questions/blob/main/Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
We are given a table called customer_state_log containing the following columns:

cust_id: The ID of the customer.
state: The state of the session, where 1 indicates the session is active and 0
indicates the session has ended.
timestamp: The timestamp when the state change occurred.

Our task is to calculate how many hours each user was active during the day based on the state transitions.


In [10]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

from pyspark.sql.window import Window

spark=SparkSession.builder.appName("learning").getOrCreate()

data = [
    ('c001', 1, '07:00:00'),
    ('c001', 0, '09:30:00'),
    ('c001', 1, '12:00:00'),
    ('c001', 0, '14:30:00'),
    ('c002', 1, '08:00:00'),
    ('c002', 0, '09:30:00'),
    ('c002', 1, '11:00:00'),
    ('c002', 0, '12:30:00'),
    ('c002', 1, '15:00:00'),
    ('c002', 0, '16:30:00'),
    ('c003', 1, '09:00:00'),
    ('c003', 0, '10:30:00'),
    ('c004', 1, '10:00:00'),
    ('c004', 0, '10:30:00'),
    ('c004', 1, '14:00:00'),
    ('c004', 0, '15:30:00'),
    ('c005', 1, '10:00:00'),
    ('c005', 0, '14:30:00'),
    ('c005', 1, '15:30:00'),
    ('c005', 0, '18:30:00')
]

columns = ["cust_id", "state", "timestamp"]

df=spark.createDataFrame(data,columns).withColumn("timestamp",col('timestamp').cast("timestamp"))

df.show()

window_spec=Window.partitionBy('cust_id').orderBy('timestamp')

df1=df.withColumn("next_timestamp",lead('timestamp').over(window_spec))

df1.show()

time_diff=((unix_timestamp(col('next_timestamp')))- (unix_timestamp(col('timestamp'))))/3600

df2=df1.filter("state==1").groupBy('cust_id').agg(sum(time_diff).alias('total_active_hours'))

df2.show()

+-------+-----+-------------------+
|cust_id|state|          timestamp|
+-------+-----+-------------------+
|   c001|    1|2025-02-01 07:00:00|
|   c001|    0|2025-02-01 09:30:00|
|   c001|    1|2025-02-01 12:00:00|
|   c001|    0|2025-02-01 14:30:00|
|   c002|    1|2025-02-01 08:00:00|
|   c002|    0|2025-02-01 09:30:00|
|   c002|    1|2025-02-01 11:00:00|
|   c002|    0|2025-02-01 12:30:00|
|   c002|    1|2025-02-01 15:00:00|
|   c002|    0|2025-02-01 16:30:00|
|   c003|    1|2025-02-01 09:00:00|
|   c003|    0|2025-02-01 10:30:00|
|   c004|    1|2025-02-01 10:00:00|
|   c004|    0|2025-02-01 10:30:00|
|   c004|    1|2025-02-01 14:00:00|
|   c004|    0|2025-02-01 15:30:00|
|   c005|    1|2025-02-01 10:00:00|
|   c005|    0|2025-02-01 14:30:00|
|   c005|    1|2025-02-01 15:30:00|
|   c005|    0|2025-02-01 18:30:00|
+-------+-----+-------------------+

+-------+-----+-------------------+-------------------+
|cust_id|state|          timestamp|     next_timestamp|
+-------+-----+--------